# Problem statement

* Input:
    - $X \in \{0, 1\}^{n\times f}$ binary matrix of participants (rows) and their attributes (columns).
    - $k < n$ integer number of desired participants.  
    - $q \in [0, 1]^f$ target Bernoulli distribution for each feature
    - $w \in \mathbb{R}_+^f$ weighting over features
 
* Goal: select a $k$-hot vector $y \in \{0, 1\}^n$ with $\|y\| = k$ such that the KL divergence between $D(E[\langle y, X\rangle ] ~\|~ q)$ is minimized.



---
### Notes

* $D(p~\|~q) := \sum_i p_i \log \frac{p_i}{q_i}$
* This seems np-hard in general, but probably approximable via submodular optimization

### Web app

* Build this in a nice little flask app with a CSV uploader, and bootstrap widgets to control $k$ and $q$
* Visualize the distribution of the solution

### Implementation

* Let $v_i$ be the indicator vector of selected rows at step $i$ of a greedy algorithm.
* The distribution for the $j$th attribute is 
$$
p_j = \frac{\langle y, X \rangle_j}{\|y\|}
$$

* The (weighted) objective function is:

$$
f(y) = - \sum_j w_j D(p_j \| q_j)
$$
where
$$
D(a \| b) = a\log \frac{a}{b} + (1-a)\log\frac{1-a}{1-b}
$$

We maximize the objective by iteratively optimizing marginal improvement:

$$
y_{t+1} \leftarrow y_t + \text{arg}\max_e f(y_t + e) - f(y_t)
$$

$$
p'_j = \frac{\langle y, X \rangle_j + \langle e, X \rangle_j }{1 + \|y\|}
= \frac{\langle y, X \rangle_j + X_{ej} }{1 + \|y\|}
= \frac{\|y\| p_j + X_{ej}}{1 + \|y\|}
$$

$$
\Delta f = - \sum_j w_j \left(D(p'_j \| q_j) - D( p_j \| q_j) \right)
$$

### Additional ideas

* Random initialization: run $T$ threads in parallel with random seeds; pick the one that achieves the best objective.
* Alternately, initialiaze by the pair of items with the minimum overlap (equivalently, maximum weighted l2 score).
* Or just use pre-selects and try to compensate with remaining selectors.

In [254]:
import numpy as np
import scipy
import numba

In [277]:
def obj(p, w, q):
    # Prevent numerical underflow in log
    
    amin = 1e-200
    
    pbar = 1. - p
    qbar = 1. - q
    
    H = p * (np.log(p + amin) - np.log(q + amin)) + pbar * (np.log(pbar + amin) - np.log(qbar + amin))
    return - H.dot(w)

def _entrofy(X, k, w=None, q=None, pre_selects=None):
    
    n_participants, n_attributes = X.shape
    
    if w is None:
        w = np.ones(n_attributes)
        
    if q is None:
        q = 0.5 * np.ones(n_attributes)
        
    assert 0 < k <= n_participants
    assert not np.any(w < 0)
    assert np.all(q >= 0.0) and np.all(q <= 1.0)
    assert len(w) == n_attributes
    assert len(q) == n_attributes
    
    if k == n_participants:
        return np.arange(n_participants)
    
    # Initialization
    y = np.zeros(n_participants, dtype=bool)
    
    if pre_selects is None:
        # Select one at random
        pre_selects = np.random.choice(n_participants, size=1)
    
    y[pre_selects] = True
    
    while y.sum() < k:
        i = y.sum()
        # Initialize the distribution vector
        p = np.nanmean(X[y], axis=0)
        
        # Compute the marginal gains
        p_new = (p * i + X) / (i + 1.0)
        delta = obj(p_new, w, q) - obj(p, w, q)
        
        # Knock out the points we've already taken
        delta[y] = -np.inf
        
        # Select the top score
        y_new = np.argmax(delta)
        y[np.argmax(delta)] = True
        
    return obj(np.nanmean(X[y], axis=0), w, q), np.flatnonzero(y)

def entrofy(X, k, w=None, q=None, pre_selects=None, n_samples=15):
    '''Entrofy your panel.
    
    Parameters
    ----------
    X : np.ndarray, shape=(n, f), dtype=bool
        Rows are participants
        Columns are attributes
        
    k : int in (0, n]
        The number of participants to select
        
    w : np.ndarray, non-negative, shape=(f,)
        Weighting over the attributes
        By default, a uniform weighting is ued
        
    q : np.darray, values in [0, 1], shape=(f,)
        Target distribution vector for the attributes.
        By default, 1/2
        
    pre_selects : None or iterable
        Optionally, you may pre-specify a set of rows to be forced into the solution.
        
    n_samples : int > 0
        If pre_selects is None, run `n_samples` random initializations and return
        the solution with the best objective value.
        
        
    Returns
    -------
    score : float
        The score of the solution found.  Larger is better.
        
    idx : np.ndarray, shape=(k,)
        Indicies of the selected rows
        
    '''
    if pre_selects is not None:
        n_samples = 1
        
    results = [_entrofy(X, k, w=w, q=q, pre_selects=pre_selects) for i in range(n_samples)]
    
    max_score, best = results[0]
    for score, solution in results[1:]:
        if score > max_score:
            max_score = score
            best = solution
            
    return max_score, best

In [412]:
X = np.random.randn(300, 5)
X = np.greater_equal(X, np.linspace(-1.5, 1.5, X.shape[1]))

In [413]:
X.shape

(300, 5)

In [421]:
score, y = entrofy(X, 25, n_samples=10)

In [422]:
score, y

(-0.010418234187507737,
 array([  0,   6,   7,  19,  23,  30,  35,  47,  50,  54,  63,  72,  84,
         93,  96, 101, 103, 132, 155, 159, 197, 202, 232, 240, 275]))

In [423]:
X[y].mean(axis=0)

array([ 0.56,  0.52,  0.48,  0.48,  0.48])

In [424]:
y

array([  0,   6,   7,  19,  23,  30,  35,  47,  50,  54,  63,  72,  84,
        93,  96, 101, 103, 132, 155, 159, 197, 202, 232, 240, 275])

In [425]:
X[y]

array([[ True,  True,  True, False, False],
       [ True, False, False, False, False],
       [ True,  True, False, False,  True],
       [ True,  True,  True, False,  True],
       [False, False, False,  True,  True],
       [False,  True,  True, False, False],
       [ True,  True, False, False,  True],
       [False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True],
       [ True,  True, False,  True,  True],
       [False,  True, False,  True, False],
       [ True, False,  True, False,  True],
       [False, False, False, False, False],
       [ True,  True, False,  True,  True],
       [ True, False,  True,  True, False],
       [ True, False, False, False,  True],
       [False, False, False,  True, False],
       [ True, False,  True, False,  True],
       [ True,  True,  True,  True, False],
       [False, False,  True, False, False],
       [False, False,  True, False, False],
       [False,  True, False,  Tr